In [1]:
from openai import OpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain.agents import tool
from utils import pretty_print_conversation, gpt_process_function_calling

c:\Users\saisa\.conda\envs\llm-agents-uplifting\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
client = OpenAI()

n_questions = 5

sys_prompt = {
    "role": "system",
    "content": "You are an agent that responds with only Python code. You will think through step by step and write correct and minimal code."
}

workflow = [
    (f"Write python code to sample {n_questions} questions from cais/MMLU dataset (use 'formal_logic' config) using 'datasets' library.", []),
    # (f"Ask all {n_questions} questions to each LLM and make each LLM respond with a choice and check their answers. Each LLM should be asked all {n_questions} with their choices.",[]),
    # ("Calculate the number of correct answers for each model and then calculate the accuracy of each model", []),
    # ("Report the accuracy of all the models. What is the best model?", [])
]

In [7]:
messages = []
messages.append(sys_prompt)
for instruction, functions in workflow:
    kwargs = {}
    if len(functions) > 0:
        functions = [convert_to_openai_tool(tool(f)) for f in functions]
        kwargs = {"tools": functions}
    messages.append({"role": "user", "content": instruction})
    output = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=messages, seed=0,
                                            **kwargs)
    output = gpt_process_function_calling(output)
    messages.append({"role": "assistant", "content": output})
pretty_print_conversation(messages)

system: You are an agent that responds with only Python code. You will think through step by step and write correct and minimal code.

user: Write python code to sample 5 questions from cais/MMLU dataset (use 'formal_logic' config) using 'datasets' library.

assistant: ```python
from datasets import load_dataset
import random

# Load the dataset with the 'formal_logic' configuration
dataset = load_dataset('cais/MMLU', 'formal_logic')

# Extract the questions from the dataset
questions = dataset['train']['question']

# Sample 5 questions randomly
sampled_questions = random.sample(questions, 5)

# Print the sampled questions
for question in sampled_questions:
    print(question)
```

